In [ ]:
%pip install openicl

# Self-adaptive In-context Learning
---
code for paper [Self-adaptive In-context Learning](https://arxiv.org/abs/2212.10375)

## Templates 

In [1]:
from openicl import PromptTemplate

# SST-2
sst2_tp_dict = {
    0 : '</E>Positive Movie Review: \"<X>\"', 
    1 : '</E>Negative Movie Review: \"<X>\"',
}
sst2_template = PromptTemplate(sst2_tp_dict, column_token_map={'text' : '<X>'}, ice_token='</E>')

# SST-5
sst5_tp_dict = {
    
}

# SNLI & MNLI
xnli_tp_dict = {
    0 : '</E><X1>? Yes, <X2>',
    1 : '</E><X1>? Maybe, <X2>',
    2 : '</E><X1>? No, <X2>'
}
xnli_template = PromptTemplate(xnli_tp_dict, column_token_map={'premise' : '<X1>', 'hypothesis' : '<X2>'}, ice_token='</E>')

# QNLI 

# TREC

# AgNews

# Commonsense QA


## Experiments

### MDL (method in this paper)

#### 3. SNLI

In [ ]:
from openicl import DatasetReader, MDLRetriever, PPLInferencer, AccEvaluator
from datasets import load_dataset

dataset = load_dataset('snli')

# If you only want to test part of the test set, you can use the following code
# dataset['test'] = dataset['test'].select(list(range(100)))

data = DatasetReader(dataset, input_columns=['premise', 'hypothesis'], output_column='label')

# Maybe we use a different seed, but the results are very closed.
# If you don't have enough memory, you can leave `batch_size` unset
retriever = MDLRetriever(data, ice_num=8, candidate_num=30, select_time=10, seed=1, batch_size=12, ice_template=xnli_template)

inferencer = PPLInferencer(model_name='gpt2-xl', batch_size=8)

predictions = inferencer.inference(retriever, ice_template=xnli_template, output_json_filename='mdl_snli')

scores = AccEvaluator().score(predictions=predictions, references=data.references)
print(scores)